In [1]:
# Ignore warnign from torch: Cuda
import warnings
warnings.filterwarnings("ignore", message="", category=Warning, module="torch")

## 🤗 Questions and Answers Generator 🐍
### Generating Questions

In [2]:
%reload_ext watermark
%watermark -uniz -a "Prayson W. Daniel" -vm -p matplotlib,numpy,torch,transformers

Author: Prayson W. Daniel

Last updated: 2020-12-16T11:43:52.295562+00:00

Python implementation: CPython
Python version       : 3.8.6
IPython version      : 7.19.0

matplotlib  : 3.3.3
numpy       : 1.19.4
torch       : 1.7.1
transformers: 4.0.1

Compiler    : GCC 8.3.0
OS          : Linux
Release     : 4.19.128-microsoft-standard
Machine     : x86_64
Processor   : 
CPU cores   : 12
Architecture: 64bit



In [3]:
import typing as t
from pathlib import Path
from transformers import PreTrainedModel
from transformers import PreTrainedTokenizer


class ModelLoader:
    """
    Downloading and Loading Hugging FaceModels 
    
    """
    
    def __init__(self, model_name:str,
                 model_directory:str,
                 tokenizer_loader:PreTrainedTokenizer,
                 model_loader:PreTrainedModel):
        
        self.model_name = Path(model_name)
        self.model_directory = Path(model_directory)
        self.model_loader = model_loader
        self.tokenizer_loader = tokenizer_loader
        
        self.save_path = self.model_directory / self.model_name
        
        if not self.save_path.exists():
            print(f'[+] {self.save_path} does not exit!')
            self.save_path.mkdir(parents=True, exist_ok=True)
            self.__download_model()         
        
        self.tokenizer, self.model = self.__load_model()
        
    def __repr__(self):
        return f'{self.__class__.__name__}(model={self.save_path})'
        
    # Download model from HuggingFace
    def __download_model(self) -> None:

        print(f'[+] Downloading {self.model_name}')
        tokenizer = self.tokenizer_loader.from_pretrained(f'{self.model_name}')
        model = self.model_loader.from_pretrained(f'{self.model_name}')

        print(f'[+] Saving {self.model_name} to {self.save_path}')
        tokenizer.save_pretrained(f'{self.save_path}')
        model.save_pretrained(f'{self.save_path}')
        
        print('[+] Process completed')
    
    # Load model
    def __load_model(self)->t.Tuple:
    
        print(f'[+] Loading model from {self.save_path}')
        tokenizer = self.tokenizer_loader.from_pretrained(f'{self.save_path}')
        model = self.model_loader.from_pretrained(f'{self.save_path}')

        print(f'[+] Loading completed')
        
        return tokenizer, model
    
    def retrieve(self):
        return self.tokenizer, self.model

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [6]:
MODEL_NAME = "mrm8488/t5-base-finetuned-question-generation-ap"
MODEL_PATH = "./ml_model"

model_loader = ModelLoader(model_name=MODEL_NAME,
                            model_directory=MODEL_PATH,
                            tokenizer_loader=AutoTokenizer,
                            model_loader=AutoModelForSeq2SeqLM
                            )

[+] Loading model from ml_model/mrm8488/t5-base-finetuned-question-generation-ap
[+] Loading completed


In [7]:
model = model_loader.model
tokenizer = model_loader.tokenizer

In [8]:
def answer_question(answer, context, max_length=64):
    input_text = f"answer: {answer}  context: {context}"
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

    return tokenizer.decode(output[0])

In [9]:
context = 'This is a tale of a tiny snail and a great big grey blue humpback whale'
answer = "tiny"
answer_question(answer, context)

'<pad> question: How big is the snail in this tale?</s>'

In [10]:
answer_question('grey', context, max_length=128)

'<pad> question: What color is the humpback whale?</s>'

In [11]:
answer_question('big', context, max_length=128)

'<pad> question: What size is the humpback whale?</s>'

In [12]:
from transformers import AutoModelForQuestionAnswering

tokenizer, model = ModelLoader(model_name="deepset/roberta-base-squad2",
                               model_directory=MODEL_PATH,
                               tokenizer_loader=AutoTokenizer,
                               model_loader=AutoModelForQuestionAnswering
                                ).retrieve()

[+] Loading model from ml_model/deepset/roberta-base-squad2
[+] Loading completed


In [13]:
def ask_question(question, context):
    
    inputs = tokenizer(question, context, 
                       add_special_tokens=True, 
                       return_tensors='pt')
    
    input_ids = inputs['input_ids'].tolist()[0]
    
    outputs = model(**inputs)
    
    answer_start = outputs.start_logits.argmax().item()
    answer_end = outputs.end_logits.argmax().item() + 1

    answer = tokenizer.decode(input_ids[answer_start:answer_end])

    return answer.strip()

In [14]:
context = 'This is a tale of a tiny snail and a great big grey blue humpback whale'

q1 = "What is the color of the humpback whale?"
q2 = "What kind of a whale is it?"

output = ask_question(q2, context)

In [15]:
output

'grey blue humpback'

In [17]:
import numpy as np

In [18]:
inputs = tokenizer(q1, context, 
                       add_special_tokens=True, 
                       return_tensors='pt')
    
input_ids = inputs['input_ids'].tolist()[0]
outputs = model(**inputs)
    


start = outputs.start_logits.detach().numpy()
end = outputs.end_logits.detach().numpy()

# Ensure padded tokens & question tokens cannot belong to the set of candidate answers.
#?? undesired_tokens = np.abs(np.array(feature.p_mask) - 1) & feature.attention_mask

# Generate mask

undesired_tokens = inputs['attention_mask']
undesired_tokens_mask = undesired_tokens == 0.0

# Make sure non-context indexes in the tensor cannot contribute to the softmax
start_ = np.where(undesired_tokens_mask, -10000.0, start)
end_ = np.where(undesired_tokens_mask, -10000.0, end)

 # Normalize logits and spans to retrieve the answer
start_ = np.exp(start_ - np.log(np.sum(np.exp(start_), axis=-1, keepdims=True)))
end_ = np.exp(end_ - np.log(np.sum(np.exp(end_), axis=-1, keepdims=True)))

# Compute the score of each tuple(start, end) to be the real answer
outer = np.matmul(np.expand_dims(start_, -1), np.expand_dims(end_, 1))

# Remove candidate with end < start and end - start > max_answer_len
max_answer_len = 15
candidates = np.tril(np.triu(outer), max_answer_len - 1)
scores_flat = candidates.flatten()

idx_sort = [np.argmax(scores_flat)]
start, end = np.unravel_index(idx_sort, candidates.shape)[1:]
end += 1
score = candidates[0, start, end-1]
start, end, score = start.item(), end.item(), score.item()


print(tokenizer.decode(input_ids[start:end]).strip())
print(score)

grey blue
0.34658950567245483


In [19]:
from transformers import pipeline

nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

QA_input = {
    'question': q1,
    'context': context
}

nlp(QA_input)

{'score': 0.3962889611721039, 'start': 47, 'end': 56, 'answer': 'grey blue'}

In [215]:
#ask_question(q1, context)

In [25]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer, model = ModelLoader(model_name='valhalla/t5-small-qa-qg-hl',
                            model_directory=MODEL_PATH,
                            tokenizer_loader=AutoTokenizer,
                            model_loader=AutoModelForSeq2SeqLM,
                            ).retrieve()

[+] Loading model from ml_model/valhalla/t5-small-qa-qg-hl
[+] Loading completed


In [26]:
input_text = "question: What answer to life? context: 42 is the answer to life, the universe and everything."

features = tokenizer([input_text], return_tensors='pt')

output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=64)

In [27]:
tokenizer.decode(output[0])

'<pad> 42</s>'

In [28]:
from transformers import AutoModelForTokenClassification
from transformers import pipeline

ner_loader = ModelLoader(model_name="dslim/bert-base-NER",
                            model_directory=MODEL_PATH,
                            tokenizer_loader=AutoTokenizer,
                            model_loader=AutoModelForTokenClassification,
                            )

tokenizer, model = ner_loader.retrieve()

[+] Loading model from ml_model/dslim/bert-base-NER
[+] Loading completed


In [29]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)
example = "My name is Prayson and I live in Copenhagen. I work at NASA"

In [30]:
nlp(example)

[{'entity_group': 'PER', 'score': 0.9993826746940613, 'word': 'P'},
 {'entity_group': 'PER', 'score': 0.8052563965320587, 'word': '##rayson'},
 {'entity_group': 'LOC', 'score': 0.9989917278289795, 'word': 'Copenhagen'},
 {'entity_group': 'ORG', 'score': 0.9983671307563782, 'word': 'NASA'}]

In [31]:
ModelLoader(model_name='valhalla/t5-small-qa-qg-hl',
                            model_directory=MODEL_PATH,
                            tokenizer_loader=AutoTokenizer,
                            model_loader=AutoModelForSeq2SeqLM,
                            )

[+] Loading model from ml_model/valhalla/t5-small-qa-qg-hl
[+] Loading completed


ModelLoader(model=ml_model/valhalla/t5-small-qa-qg-hl)

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [4]:
import requests
requests.get('https://www.google.com')

ConnectionError: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f90d87a2160>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [8]:
MODEL_NAME = "valhalla/t5-base-e2e-qg"
MODEL_PATH = "./ml_model"
tokenizer, model = ModelLoader(model_name=MODEL_NAME,
                            model_directory=MODEL_PATH,
                            tokenizer_loader=AutoTokenizer,
                            model_loader=AutoModelForSeq2SeqLM,
                            ).retrieve()

file ml_model/valhalla/t5-base-e2e-qg/config.json not found


[+] Loading model from ml_model/valhalla/t5-base-e2e-qg


OSError: Can't load config for 'ml_model/valhalla/t5-base-e2e-qg'. Make sure that:

- 'ml_model/valhalla/t5-base-e2e-qg' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'ml_model/valhalla/t5-base-e2e-qg' is the correct path to a directory containing a config.json file

